In [ ]:
#!pip install opencv-python==3.4.2.16


In [ ]:
#!pip install opencv-contrib-python==3.4.2.16



In [ ]:
import cv2

In [ ]:
#checks if SIFT is working
sift = cv2.xfeatures2d.SIFT_create()
print(sift)

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
import glob
import os
import sys
import cv2
import numpy as np


In [ ]:
#modify function to extract result size of image and get the coordinates
# Use the keypoints to stitch the images
#grab image name alsso easier for writing
def get_stitched_image(img1, img2, M):

    # Get width and height of input images
    w1,h1 = img1.shape[:2]
    w2,h2 = img2.shape[:2]
    
    
    # Get the canvas dimesions
    img1_dims = np.float32([ [0,0], [0,w1], [h1, w1], [h1,0] ]).reshape(-1,1,2)
    img2_dims_temp = np.float32([ [0,0], [0,w2], [h2, w2], [h2,0] ]).reshape(-1,1,2)


    # Get relative perspective of second image
    img2_dims = cv2.perspectiveTransform(img2_dims_temp, M)

    # Resulting dimensions
    result_dims = np.concatenate( (img1_dims, img2_dims), axis = 0)

    # Getting images together
    # Calculate dimensions of match points
    [x_min, y_min] = np.int32(result_dims.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(result_dims.max(axis=0).ravel() + 0.5)
    

    # Create output array after affine transformation 
    transform_dist = [-x_min,-y_min]
    #show tha image 
    
    #plt.imshow(img2)
    transform_array = np.array([[1, 0, transform_dist[0]], 
                                [0, 1, transform_dist[1]], 
                                [0,0,1]]) 

    # Warp images to get the resulting image
    result_img = cv2.warpPerspective(img2, transform_array.dot(M), 
                                    (x_max-x_min, y_max-y_min))
    #Result_image1
    #selective image region print 
    res1 = result_img[transform_dist[1]:w1+transform_dist[1],transform_dist[0]:h1+transform_dist[0]]
    
    
    #image stitched final
    result_img[transform_dist[1]:w1+transform_dist[1], 
                transform_dist[0]:h1+transform_dist[0]] = img1
    


    #extract image2 shape since it has been pasted on the canvas
    w1,h1,_ = img2.shape
    
    #extract reusult image height and width for cropping
    w2,h2,_ = result_img.shape
    

    print('Stitching Done')

    
    return result_img


In [ ]:

# Find SIFT and return Homography Matrix
def get_sift_homography(img1, img2):
     
    sift = cv2.xfeatures2d.SIFT_create()
    
    # Extract keypoints and descriptors
    k1, d1 = sift.detectAndCompute(img1, None)
    k2, d2 = sift.detectAndCompute(img2, None)
    if ( d1 is None or d2 is None):
      print('Images are different')
      print('Set of images from _ to ', image_name)
      print(d1)
      
    else:

      FLANN_INDEX_KDTREE = 0
      index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
      search_params = dict(checks=50)   # or pass empty dictionary

      flann = cv2.FlannBasedMatcher(index_params,search_params)

      matches = flann.knnMatch(d1,d2,k=2)


      # Make sure that the matches are good
      #verify_ratio = 0.6 # Source: stackoverflow
      verified_matches = []
      for m1,m2 in matches:
          # Add to array only if it's a good match
          if m1.distance < 0.85 * m2.distance:
              verified_matches.append(m1)

      # Mimnum number of matches
      min_matches = 8
      if len(verified_matches) > min_matches:

          # Array to store matching points
          img1_pts = []
          img2_pts = []

          # Add matching points to array
          for match in verified_matches:
              img1_pts.append(k1[match.queryIdx].pt)
              img2_pts.append(k2[match.trainIdx].pt)
          img1_pts = np.float32(img1_pts).reshape(-1,1,2)
          img2_pts = np.float32(img2_pts).reshape(-1,1,2)

          # Compute homography matrix
          M, mask = cv2.findHomography(img1_pts, img2_pts, cv2.RANSAC, 5.0)
          return M


In [ ]:

# Equalize Histogram of Color Images
def equalize_histogram_color(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img


In [ ]:
#resize images before sending 
#res 640 x 480

cv_img = []
img_name_list = []

#pick images from directory and add to a array. Extract filename from directory and add them to a list

#files = [item for sublist in [glob.glob(path + ext) for ext in ["/*.jpg", "/*.JPG", "/*.png", "/*.PNG"]] for item in sublist]

#print(files)
output_path = '/home/sameer/Jupyter_Notebook/Demo_Data/Output_Stitched2/'

dict_1 = {}

#combined is masked images and Images contains original images 
pathname1 = '/home/sameer/Jupyter_Notebook/Demo_Data/Sorted_Images/'


count = 0

for item1 in sorted(os.listdir(pathname1)):    
            if item1.endswith('.jpg'):
            
                path1 = os.path.join(pathname1,item1)

                #path2 = os.path.join(pathname2,item1)
                #read image 1 and 2
                count = count +1

                #read image 1 and 2
                image1 = cv2.imread(path1)
                cv_img.append(image1)
                img_name_list.append(item1)
                #print(item1)
    
img1 = cv_img[0]
count1 = 0 
previous_height = 0
set_count = 1

set_file_names = []
for i in range(0,len(cv_img)):
    
    
    img2= cv_img[i]
    
    
    image_name = img_name_list[i]
    img1 = equalize_histogram_color(img1)
    img2 = equalize_histogram_color(img2)

    M =  get_sift_homography(img1, img2)
        

    result_image = get_stitched_image(img2, img1, M)
    
    output_image = os.path.join(output_path, image_name)
    
    cv2.imwrite(output_image, result_image)
    
    img1=result_image
    
    